In [3]:
import csv
import datetime
import numpy
import os
import yaml
import math
import numpy as np
from itertools import islice
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.multi import  MultiEncoder
from  nupic.encoders.scalar import ScalarEncoder
from nupic.encoders.category import CategoryEncoder
from nupic.algorithms.sdr_classifier import  SDRClassifier
from time import  time
import json
import urllib2
import capnp
from nupic.algorithms.spatial_pooler import SpatialPooler
import ipywidgets as wg
from IPython.display import display
import os

class Prediction():
    
    def __init__(self):
        #Baseline for lesarning
        self.sink_tap = []
        self.shower = []
        self.toilet = []
        self.pressure = []
        self.flow_rate = [] 
        self.encoding = [] 
        self.val_massive = [] 
        self.pressure = []
        self.events = dict()
        self.baseline = []
        self.encoding_csv = []
        self.flow = list()
        #Result information
        self.classifier_counter = 0 
        self.event = []
        self.result_testing = [0]
        self.result_testing1 = [0]
        self.result_testing2 = [0]
        self.baseline_test = []
        self.event10 = []
        self.length = 0 
        self.filename1  = datetime.datetime.strftime(datetime.datetime.now(), "%Y.%m.%d_%H:%M:%S")
        self.b = 0
        self.t = 0
        #Array for divorce wmId
        self.list_id = [] 
        self.flag = 0 
        self.len = 0
        #Baseline for real data 
        self.wmId  = []
        self.pres1 = []
        self.wmId  = []
        self.wmId1 = []
        self.id    = []
        self.flow1 = []
        self.time1 = []
        self.length_list_id = 0

        self.label = []
        self.labels = []
        self.pred_3 = []
        self.pred_2 = []
        self.pred_1 = []
        
    def parse_dataset(self):
        self.event_all = []
        self.ev = dict()
        INPUT_DATSET = (".../real_dataset2.csv")
        with open(INPUT_DATSET) as fin:
            reader = csv.reader(fin, delimiter=',')
            reader.next()
            reader.next()
            reader.next()
            
        
            for row in reader:
                self.pressure.append(float(row[1]))
                self.flow_rate.append(float(row[2]))
                self.event.append(row[3])
            for tt in range(0,20000):
                self.event_all.append(self.event[tt])
            kr = 0    
            for cur in xrange(len(self.event_all)): # cur - current self.infex in massive event
                kr = map(int,self.event[cur].split(",")) #  Parse string
                for tir in xrange(len(kr)):
                    additional_sink_tap = 0
                    additional_toilet = 0
                    additional_shower = 0 
                    if kr[tir] == 2: 
                        additional_toilet = 1
                    if kr[tir] == 1: 
                        additional_sink_tap = 1
                    if kr[tir] == 3: 
                        additional_shower = 1
						
                self.ev['additional_toilet'] = additional_toilet
                self.ev['additional_shower'] = additional_shower
                self.ev['additional_sink_tap'] = additional_sink_tap
    
                if tir == 0:
                    self.events['toilet'] = additional_toilet
                    self.events['shower']  = additional_shower 
                    self.events['sink_tap'] = additional_sink_tap

                self.sink_tap.append(self.events['sink_tap'])
                self.shower.append(self.events['shower'])
                self.toilet.append(self.events['toilet'])

    def baseline_learning (self,left_r):
        self.val_massive.append(self.pressure[left_r])
        if(len(self.val_massive)>1):
            for i in range(2,len(self.pressure)):
                self.sum_sred = math.ceil(sum(self.val_massive)/len(self.val_massive))
                enc_value = (self.sum_sred - self.val_massive[-1])
                sq = math.sqrt(enc_value**2)

                if ((self.flow_rate[left_r + i-1] - self.flow_rate[left_r + i - 2])<0.5 and (self.flow_rate[left_r + i-2] - self.flow_rate[left_r + i - 1])<0.5):
                    print "Flow_rate-1", self.flow_rate[left_r + i - 1] - self.flow_rate[left_r + i - 2]
                else:
                    del self.val_massive[-1]
                    try:
                        self.sum_sred_1 = sum(self.val_massive) / len(self.val_massive)
                    except ZeroDivisionError:
                        pass
                    for vts in range(0, len(self.val_massive)):
                        positive_value = self.sum_sred_1 - self.val_massive[vts]
                        self.encoding.append(round(positive_value, 2))
                        self.baseline.append(round(self.sum_sred_1, 2))
                    break

        else:
            self.val_massive.append(self.pressure[left_r+1])

            for i in xrange(2,len(self.pressure)):
                self.sum_sred = math.ceil(sum(self.val_massive) / len(self.val_massive))
                enc_value = self.sum_sred - self.val_massive[-1]
                sq = math.sqrt(enc_value**2)

                if ((self.flow_rate[left_r + i - 1] - self.flow_rate[left_r + i - 2]) < 0.5 and (self.flow_rate[left_r + i - 2] - self.flow_rate[left_r + i - 1]) < 0.5):
                    self.val_massive.append(self.pressure[left_r+i])
                else:
                    del  self.val_massive[-1]
                    try:
                        self.sum_sred_1 = sum(self.val_massive) / len(self.val_massive)
                    except ZeroDivisionError:
                        pass

                    for vts in range(0, len(self.val_massive)):
                        positive_value = self.sum_sred_1 - self.val_massive[vts]
                        self.encoding.append(round(positive_value, 2))
                        self.baseline.append(round(self.sum_sred_1, 2))
                    break
    
        self.index = i +left_r
        self.val_massive = []

    def calling_diff(self):
        ty = 1
        self.baseline_learning(ty)
        while (self.index <19980):
            self.baseline_learning(self.index-1)
        headers = ("flow","event","sink_tap","toilet","shower","encoding","baseline","pressure")
        type = ("float","int","int","int","int","float","float","float")
        meta = (" "," "," "," "," ")
        with open('test.csv', 'w') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(headers)
            csv_writer.writerow(type)
            csv_writer.writerow(meta)
            for pot in range(0, 19970):
                csv_writer.writerow([self.flow_rate[pot],self.event[pot],self.sink_tap[pot],self.toilet[pot],self.shower[pot],self.encoding[pot],self.baseline[pot],self.pressure[pot]])
    
    def open_params(self):
        
        PARAMS_PATH =  (".../model_iot.yaml")
        with open(PARAMS_PATH, "r") as f:
            modelParams = yaml.safe_load(f)["modelParams"]   
            self.spParams = modelParams["spParams"]
            self.tmParams = modelParams["tmParams"]
            self.clParams = modelParams["clParams"]
        self.sink_tap_Encoder = ScalarEncoder(name="sink_tap", w=7, n=14, minval=0, maxval=1,forced=True)
        self.toilet_Encoder = ScalarEncoder(name="toilet", w=7, n=14, minval=0, maxval=1,forced=True)
        self.shower_Encoder = ScalarEncoder(name="shower", w=7, n=14, minval=0, maxval=1,forced=True)

        self.baselineEncoder = ScalarEncoder(name = "baseline",w = 21, n = 2625,minval= 47,maxval=75,forced= True)
        self.pressEncoder = ScalarEncoder(name = "pressure",w = 21, n = 462,minval= 44,maxval=66,forced= True)

        self.flowEncoder = ScalarEncoder(name="flow", w=15, n=1050, minval=0, maxval=6,forced = True)
        self.encodingWidth = (self.sink_tap_Encoder.getWidth()+self.flowEncoder.getWidth()+self.baselineEncoder.getWidth()
                                +self.toilet_Encoder.getWidth()+self.flowEncoder.getWidth()+self.baselineEncoder.getWidth()
                                +self.shower_Encoder.getWidth()+self.flowEncoder.getWidth()+self.baselineEncoder.getWidth())

        self.sp = SpatialPooler(
            inputDimensions=(self.encodingWidth,),
            columnDimensions=(self.spParams["columnCount"],),
            potentialPct=self.spParams["potentialPct"],
            potentialRadius=self.encodingWidth,
            globalInhibition=self.spParams["globalInhibition"],
            localAreaDensity=self.spParams["localAreaDensity"],
            numActiveColumnsPerInhArea=self.spParams["numActiveColumnsPerInhArea"],
            synPermInactiveDec=self.spParams["synPermInactiveDec"],
            synPermActiveInc=self.spParams["synPermActiveInc"],
            synPermConnected=self.spParams["synPermConnected"],
            boostStrength=self.spParams["boostStrength"],
            seed=self.spParams["seed"],
            wrapAround=True
        )

        self.tm = TemporalMemory(
            columnDimensions=(self.tmParams["columnCount"],),
            cellsPerColumn=self.tmParams["cellsPerColumn"],
            activationThreshold=self.tmParams["activationThreshold"],
            initialPermanence=self.tmParams["initialPerm"],
            connectedPermanence=self.spParams["synPermConnected"],
            minThreshold=self.tmParams["minThreshold"],
            maxNewSynapseCount=self.tmParams["newSynapseCount"],
            permanenceIncrement=self.tmParams["permanenceInc"],
            permanenceDecrement=self.tmParams["permanenceDec"],
            predictedSegmentDecrement=self.tmParams["predictedSegmentDecrement"],
            maxSegmentsPerCell=self.tmParams["maxSegmentsPerCell"],
            maxSynapsesPerSegment=self.tmParams["maxSynapsesPerSegment"],
            seed=self.tmParams["seed"]
        )

        self.classifier = SDRClassifier(
            steps  = [1],alpha=self.clParams["alpha"],verbosity= self.clParams["verbosity"]
        )
        self.classifier1 = SDRClassifier(
            steps  = [1],alpha=self.clParams["alpha"],verbosity= self.clParams["verbosity"]
        )
        self.classifier2 = SDRClassifier(
            steps  = [1],alpha=self.clParams["alpha"],verbosity= self.clParams["verbosity"]
        )

    def runLearning(self,numRecords):
        learning_time = time()
        with open("test.csv", "r") as fin:
            reader = csv.reader(fin)
            headers = reader.next()
            reader.next()
            reader.next()
            learning_progress = wg.IntProgress(min = 0,max = numRecords,value = 0,description = " Learning loading",step = 1,bar_style='info')
            display(learning_progress)
            for count, record in enumerate(reader):
                #print "Count",count
                learning_progress.value = count
                if count >= numRecords: break

                # Convert data value string into float.
                self.sink_tap_value = float(record[2]) # device 1
                self.toilet_value = float(record[3]) #device 2
                self.shower__value = float(record[4]) # device 3

                 # event_value_7 = float(record[8]) # device 7
                self.baseline_all = float(record[6])
                self.flow_value  = float(record[0])
                # To encode, we need to provide zero-filled numpy arrays for the encoders
                # to populate.
                self.eventBits = numpy.zeros(self.sink_tap_Encoder.getWidth())
                self.eventBits_2 = numpy.zeros(self.toilet_Encoder.getWidth())
                self.eventBits_3 = numpy.zeros(self.shower_Encoder.getWidth())

                self.baseline_Bits = numpy.zeros(self.baselineEncoder.getWidth())
                self.flowBits = numpy.zeros(self.flowEncoder.getWidth())

                 # Now we call the encoders to create bit representations for each value.
                self.sink_tap_Encoder.encodeIntoArray(self.sink_tap_value,self.eventBits)
                self.toilet_Encoder.encodeIntoArray(self.toilet_value,self.eventBits_3)
                self.shower_Encoder.encodeIntoArray(self.shower__value,self.eventBits_2) 

                self.baselineEncoder.encodeIntoArray(self.baseline_all,self.baseline_Bits)
                self.flowEncoder.encodeIntoArray(self.flow_value,self.flowBits)

                # Concatenate all these encodings into one large encoding for Spatial
                 # Pooling.
                self.encoding = numpy.concatenate(
                    [self.eventBits,self.flowBits,self.baseline_Bits,self.eventBits_2,self.flowBits,self.baseline_Bits,self.eventBits_3,self.flowBits,self.baseline_Bits]
                )

                # Create an array to represent active columns, all initially zero. This
                 # will be populated by the compute method below. It must have the same
                 # dimensions as the Spatial Pooler.
                self.activeColumns = numpy.zeros(self.spParams["columnCount"])
                 # activeColumns1 = numpy.zeros(self.spParams["columnCount"])

                 # Execute Spatial Pooling algorithm over input space.

                self.sp.compute(self.encoding,True,self.activeColumns)

                # sp.compute(encoding1, True, activeColumns)

                self.activeColumnIndices = numpy.nonzero(self.activeColumns)[0]

                 # Execute Temporal Memory algorithm over active mini-columns.
                self.tm.compute(self.activeColumnIndices, learn=True)

                self.activeCells = self.tm.getActiveCells()

                # Get the bucket info for this input value for classification.
                self.bucketIdx_sink_tap  =  self.sink_tap_Encoder.getBucketIndices(self.sink_tap_value)[0] 
                self.bucketIdx_toilet = self.toilet_Encoder.getBucketIndices(self.toilet_value)[0]
                self.bucketIdx_shower = self.shower_Encoder.getBucketIndices(self.shower__value)[0]

                # Run classifier to translate active cells back to scalar value.
                self.classifierResult = self.classifier.compute(
                    recordNum=self.classifier_counter,
                    patternNZ=self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_sink_tap,
                        "actValue": self.sink_tap_value
                    },
                    learn=True,
                    infer=False
                )
                self.classifierResult1 = self.classifier1.compute(
                    recordNum=self.classifier_counter,
                    patternNZ=self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_toilet,
                        "actValue": self.toilet_value
                    },
                    learn=True,
                    infer=False
                )

                self.classifierResult2 = self.classifier2.compute(
                    recordNum=self.classifier_counter,
                    patternNZ=self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_shower,
                        "actValue": self.shower__value
                    },
                    learn=True,
                    infer=False
                )
                self.classifier_counter += 1
                learning_time_end = time()
 
    def runTesting(self,numRecords):       
        testing_time = time()
        REAL_DATSET = ("test.csv")
        with open(REAL_DATSET, "r") as fin:
            reader = csv.reader(fin)
            headers = reader.next()
            reader.next()
            reader.next()
            testing_progress = wg.IntProgress(min = 0,max = numRecords,value = 0,description = " Testing loading",step = 1,bar_style='info')
            display(testing_progress)
            for count, record in enumerate(reader):
                testing_progress.value = count
                if count >= numRecords: break

                # Convert data string into Python date object.
                #dateString = datetime.datetime.strptime(record[0], "%m/%d/%y %H:%M")
                # Convert data value string into float.
                self.sink_tap_value = self.result_testing[count]
                self.toilet_value = self.result_testing1[count]
                self.shower__value = self.result_testing2[count]

                self.baseline_all = float(record[6])
                self.flow_value  = float(record[0])

                # To encode, we need to provide zero-filled numpy arrays for the encoders
                # to populate.
                self.eventBits = numpy.zeros(self.sink_tap_Encoder.getWidth())
                self.eventBits_2 = numpy.zeros(self.toilet_Encoder.getWidth())
                self.eventBits_3 = numpy.zeros(self.shower_Encoder.getWidth())
 
                self.flowBits = numpy.zeros(self.flowEncoder.getWidth())
                self.baseline_Bits = numpy.zeros(self.baselineEncoder.getWidth())

                # Now we call the encoders to create bit representations for each value.
                self.sink_tap_Encoder.encodeIntoArray(self.sink_tap_value,self.eventBits)
                self.toilet_Encoder.encodeIntoArray(self.toilet_value,self.eventBits_2)
                self.shower_Encoder.encodeIntoArray(self.shower__value,self.eventBits_3)

                self.baselineEncoder.encodeIntoArray(self.baseline_all,self.baseline_Bits)
                self.flowEncoder.encodeIntoArray(self.flow_value,self.flowBits)

                # Concatenate all these encodings into one large encoding for Spatial
                # Pooling.
                self.encoding = numpy.concatenate(
                    [self.eventBits,self.flowBits,self.baseline_Bits,self.eventBits_2,self.flowBits,self.baseline_Bits,self.eventBits_3,self.flowBits,self.baseline_Bits]
                )
    
                # Create an array to represent active columns, all initially zero. This
                # will be populated by the compute method below. It must have the same
                # dimensions as the Spatial Pooler.
                self.activeColumns = numpy.zeros(self.spParams["columnCount"])

                # Execute Spatial Pooling algorithm over input space.
                self.sp.compute(self.encoding, False, self.activeColumns)

                self.activeColumnIndices = numpy.nonzero(self.activeColumns)[0]

                # Execute Temporal Memory algorithm over active mini-columns.
                self.tm.compute(self.activeColumnIndices, learn=False)

                self.activeCells = self.tm.getActiveCells()

                # Get the bucket info for this input value for classification.
                self.bucketIdx_sink_tap  = self.sink_tap_Encoder.getBucketIndices(self.sink_tap_value)[0] 
                self.bucketIdx_toilet = self.toilet_Encoder.getBucketIndices(self.toilet_value)[0]
                self.bucketIdx_shower = self.shower_Encoder.getBucketIndices(self.shower__value)[0]

      # Run classifier to translate active cells back to scalar value.
                self.classifierResult = self.classifier.compute(
                    recordNum=self.classifier_counter,
                    patternNZ=self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_sink_tap,
                        "actValue": self.sink_tap_value
                    },
                    learn=False,
                    infer=True
                )

                self.classifierResult1 = self.classifier1.compute(
                    recordNum=self.classifier_counter,
                    patternNZ= self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_toilet,
                        "actValue": self.toilet_value
                    },
                    learn=False,
                    infer=True
                )

                self.classifierResult2 = self.classifier2.compute(
                    recordNum=self.classifier_counter,
                    patternNZ= self.activeCells,
                    classification={
                        "bucketIdx": self.bucketIdx_shower,
                        "actValue": self.shower__value
                    },
                    learn=False,
                    infer=True
                )
                self.classifier_counter += 1
                 # Print the best prediction for 1 step out.
                oneStepConfidence, oneStep = sorted(
                    zip(self.classifierResult[1], self.classifierResult["actualValues"]),
                    reverse=True
                )[0]
                oneStepConfidence1, oneStep1 = sorted(
                    zip(self.classifierResult1[1], self.classifierResult1["actualValues"]),
                    reverse=True
                )[0]

                oneStepConfidence2, oneStep2 = sorted(
                    zip(self.classifierResult2[1], self.classifierResult2["actualValues"]),
                    reverse=True
                )[0]               
                testing_time_end = time()
                #print "Time testing", (testing_time_end - testing_time)

                self.result_testing.append(oneStep)
                self.result_testing1.append(oneStep1)
                self.result_testing2.append(oneStep2)
            with open("learning_dataset.csv", 'w') as csv_file:
                csv_writer = csv.writer(csv_file)
                headers = ("prediction_sink_tap","sink_tap","prediction_toilet","toilet","prediction_shower","shower","encoding", "baseline", "flow", "pressure")
                csv_writer.writerow(headers)
                for l in xrange(numRecords) :
                    csv_writer.writerow([self.result_testing[l],self.sink_tap[l],self.result_testing1[l],self.toilet[l],self.result_testing2[l],self.shower[l],self.encoding[l], self.baseline[l], self.flow_rate[l], self.pressure[l]])
            testing_time_end = time()





if __name__ == "__main__":
    test = Prediction()
    test.open_params()
    test.parse_dataset()
    test.calling_diff()
    test.runLearning(200)
    test.runTesting(100)


IOError: [Errno 2] No such file or directory: '.../model_iot.yaml'